In [ ]:
CREATE DATABASE aemet;

CREATE TABLE indicativos (
    indicativo VARCHAR(10),
    nombre VARCHAR(64),
    PRIMARY KEY (indicativo)
);

CREATE TABLE observaciones_cadiz (
    id SERIAL,
    fecha TIMESTAMP,
    nombre VARCHAR(50),
    indicativo VARCHAR(10),
    provincia VARCHAR(50),
    altitud INT,
    tmin FLOAT,
    tmax FLOAT,
    tmed FLOAT,
    prec FLOAT,
    velmedia FLOAT,
    racha FLOAT,
    hrMedia FLOAT,
    hrMax FLOAT,
    hrMin FLOAT,
    dir FLOAT,
    PRIMARY KEY (id),
    FOREIGN KEY (indicativo) REFERENCES indicativos (indicativo)
);
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a278b404-3bee-42c2-b3f0-cc12c4579d3d",
   "metadata": {},
   "outputs": [],
   "source": [
    "import json\n",
    "import boto3\n",
    "import pandas as pd\n",
    "import psycopg2\n",
    "from io import BytesIO\n",
    "\n",
    "# Configuración\n",
    "s3 = boto3.client(\"s3\")\n",
    "\n",
    "# Variables de entorno\n",
    "DB_HOST = \"tu_host_postgres\"\n",
    "DB_NAME = \"aemet\"\n",
    "DB_USER = \"postgre\"\n",
    "DB_PASS = \"password\"\n",
    "DB_PORT = 5432\n",
    "\n",
    "def lambda_handler(event, context):\n",
    "    bucket = event['Records'][0]['s3']['bucket']['name']\n",
    "    key    = event['Records'][0]['s3']['object']['key']\n",
    "    \n",
    "    print(f\"Procesando archivo: s3://{bucket}/{key}\")\n",
    "    \n",
    "    try:\n",
    "        # Descargar JSON desde S3\n",
    "        obj = s3.get_object(Bucket=bucket, Key=key)\n",
    "        data = json.loads(obj['Body'].read().decode('utf-8'))\n",
    "        \n",
    "        df = pd.DataFrame(data)\n",
    "        \n",
    "        # Convertir columnas a numérico\n",
    "        numeric_cols = [\"altitud\",\"tmed\",\"prec\",\"tmin\",\"tmax\",\n",
    "                        \"velmedia\",\"racha\",\"hrMedia\",\"hrMax\",\"hrMin\",\"dir\"]\n",
    "        \n",
    "        for c in numeric_cols:\n",
    "            if c in df.columns:\n",
    "                df[c] = pd.to_numeric(df[c], errors='coerce')\n",
    "        \n",
    "        # Conectar a PostgreSQL\n",
    "        conn = psycopg2.connect(\n",
    "            host=DB_HOST,\n",
    "            dbname=DB_NAME,\n",
    "            user=DB_USER,\n",
    "            password=DB_PASS,\n",
    "            port=DB_PORT\n",
    "        )\n",
    "        cursor = conn.cursor()\n",
    "        \n",
    "        # Insertar fila por fila \n",
    "        for _, row in df.iterrows():\n",
    "            cursor.execute(\"\"\"\n",
    "                INSERT INTO observaciones_cadiz \n",
    "                (fecha, nombre, indicativo, provincia, altitud,\n",
    "                 tmin, tmax, tmed, prec, velmedia, racha, hrMedia, hrMax, hrMin, dir)\n",
    "                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)\n",
    "                ON CONFLICT (fecha) DO NOTHING\n",
    "            \"\"\", (\n",
    "                row.get('fecha'), row.get('nombre'), row.get('indicativo'), row.get('provincia'),\n",
    "                row.get('altitud'), row.get('tmin'), row.get('tmax'), row.get('tmed'),\n",
    "                row.get('prec'), row.get('velmedia'), row.get('racha'), row.get('hrMedia'),\n",
    "                row.get('hrMax'), row.get('hrMin'), row.get('dir')\n",
    "            ))\n",
    "        \n",
    "        conn.commit()\n",
    "        cursor.close()\n",
    "        conn.close()\n",
    "        \n",
    "        print(f\" Archivo cargado en PostgreSQL: {key}\")\n",
    "        return {\"status\": \"success\"}\n",
    "    \n",
    "    except Exception as e:\n",
    "        print(\" Error:\", e)\n",
    "        return {\"status\": \"error\", \"message\": str(e)}\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:base] *",
   "language": "python",
   "name": "conda-base-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
